# FAISS Memory Study

This notebook tries to measure the memory consumption and speed of faiss in CPU and GPU


In [1]:
import numpy as np
import faiss

In [2]:
# Multiple GPUs
ngpus = faiss.get_num_gpus()

print("number of GPUs:", ngpus)

number of GPUs: 2


In [3]:
d=128
N = 100000
emb = np.array(np.random.randint(0,2, size=(N, d)), dtype=np.float32)

In [4]:
emb.shape

(100000, 128)

In [5]:
index_flat_l2 = faiss.IndexFlatL2(d)

In [6]:
index_flat_l2.train(emb)

In [7]:
index_flat_l2.add(emb)

In [8]:
npdec = np.array(np.random.randn(10000,128), dtype=np.float32)

In [9]:
npdec.shape

(10000, 128)

In [10]:
npdec.dtype

dtype('float32')

In [11]:
index_flat_l2.d

128

In [12]:
index_flat_l2.ntotal

100000

In [13]:
%%time
k=2
D,I = index_flat_l2.search(npdec, k)

CPU times: user 49.7 s, sys: 15.8 s, total: 1min 5s
Wall time: 8.87 s


In [14]:
res = faiss.StandardGpuResources()

In [15]:
res?

Type:        StandardGpuResources
String form: <faiss.swigfaiss.StandardGpuResources; proxy of <Swig Object of type 'faiss::gpu::StandardGpuResources *' at 0x7f35db1a1ae0> >
File:        ~/venv3/lib/python3.7/site-packages/faiss-1.6.3-py3.7.egg/faiss/swigfaiss.py
Docstring:   <no docstring>


In [16]:
res

<faiss.swigfaiss.StandardGpuResources; proxy of <Swig Object of type 'faiss::gpu::StandardGpuResources *' at 0x7f35db1a1ae0> >

In [17]:
# Using one GPU
res = faiss.StandardGpuResources()  # use a single GPU
# build a flat (CPU) index
index_flat = faiss.IndexFlatL2(d)

In [18]:
# make it into a gpu index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

In [19]:
res = faiss.StandardGpuResources()  # use a single GPU
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat_l2)

In [20]:
faiss.index_cpu_to_gpu?

Signature: faiss.index_cpu_to_gpu(*args)
Docstring: <no docstring>
File:      ~/venv3/lib/python3.7/site-packages/faiss-1.6.3-py3.7.egg/faiss/__init__.py
Type:      function


In [21]:
index_flat_l2?

Type:        IndexFlatL2
String form: <faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f35db1a16c0> >
File:        ~/venv3/lib/python3.7/site-packages/faiss-1.6.3-py3.7.egg/faiss/swigfaiss.py
Docstring:   <no docstring>


In [22]:
%%time
k=20
D,I = gpu_index_flat.search(npdec, k)

CPU times: user 38 ms, sys: 4.19 ms, total: 42.2 ms
Wall time: 44.4 ms


TODO check how to make it more memory efficient using float16 instead

* https://github.com/facebookresearch/faiss/issues/666
* https://github.com/facebookresearch/faiss/blob/master/gpu/utils/Float16.cu
* https://github.com/facebookresearch/faiss/issues/590

Use "lossy" indices (float16, int8, int4), instead of a complete one, this will save in space even if the computations are done in float32

In [23]:
faiss.index_cpu_to_gpu?

Signature: faiss.index_cpu_to_gpu(*args)
Docstring: <no docstring>
File:      ~/venv3/lib/python3.7/site-packages/faiss-1.6.3-py3.7.egg/faiss/__init__.py
Type:      function


Indices that I should try with the codes:

- faiss.IndexScalarQuantizer with (QT_8bit/QT_6bit/QT_4bit)
- faiss.IndexPQ

the construction can be done with index_factory: 

    index = faiss.index_factory(d, "IVF100,PQ8")
    
    
To analyze a matrix, print

    MatrixStats(my_matrix).comments (Python)

In [31]:
# finite precision ERROR on BIG numbers

# example from: https://gist.github.com/mdouze/efc94c57e2302469287b9d1a2501d277

import numpy as np

a = np.array([12345], dtype='float32')
b = np.array([12343], dtype='float32')

diff = (a * a).sum() + (b * b).sum() - 2 * np.dot(a, b)

In [27]:
diff

0.0

In [28]:
(a * a).sum(), (b * b).sum(),  2 * np.dot(a, b)

(152399020.0, 152349650.0, 304748672.0)

In [29]:
(a * a).sum() + (b * b).sum()

304748670.0

In [30]:
2 * np.dot(a, b)

304748672.0